In [1]:
# 192.168.1.35
import cv2
import mediapipe as mp
from flask import Flask, Response

# Initialisation de Flask
app = Flask(__name__)

# URL du flux ESP32-CAM
ESP32_STREAM_URL = "http://192.168.1.35:81/stream"  # Remplacez XX par l'IP de l'ESP32-CAM

# Initialisation de Mediapipe
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

# Fonction de calcul pour détecter les clignements
def calculate_blink_ratio(landmarks, left_eye_indices, right_eye_indices):
    def get_distance(p1, p2):
        return ((p1.x - p2.x) ** 2 + (p1.y - p2.y) ** 2) ** 0.5

    # Calculer les distances pour les deux yeux
    left_eye_top = landmarks[left_eye_indices[1]]
    left_eye_bottom = landmarks[left_eye_indices[5]]
    left_eye_left = landmarks[left_eye_indices[0]]
    left_eye_right = landmarks[left_eye_indices[3]]

    right_eye_top = landmarks[right_eye_indices[1]]
    right_eye_bottom = landmarks[right_eye_indices[5]]
    right_eye_left = landmarks[right_eye_indices[0]]
    right_eye_right = landmarks[right_eye_indices[3]]

    left_vertical = get_distance(left_eye_top, left_eye_bottom)
    left_horizontal = get_distance(left_eye_left, left_eye_right)

    right_vertical = get_distance(right_eye_top, right_eye_bottom)
    right_horizontal = get_distance(right_eye_left, right_eye_right)

    left_ratio = left_vertical / left_horizontal
    right_ratio = right_vertical / right_horizontal

    return (left_ratio + right_ratio) / 2

# Générer les frames pour le flux vidéo
def generate_frames():
    cap = cv2.VideoCapture(ESP32_STREAM_URL)
    left_eye_indices = [33, 160, 158, 133, 153, 144]  # Indices des points de l'œil gauche
    right_eye_indices = [362, 385, 387, 263, 373, 380]  # Indices des points de l'œil droit

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Conversion en RGB pour Mediapipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_frame)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # Calculer le ratio de clignement
                blink_ratio = calculate_blink_ratio(
                    face_landmarks.landmark, left_eye_indices, right_eye_indices
                )

                # Vérification du clignement
                blink_status = "Eyes Closed" if blink_ratio < 0.2 else "Eyes Open"

                # Ajouter du texte à l'image
                cv2.putText(frame, blink_status, (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        _, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

# Route pour le flux vidéo
@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

# Lancer le serveur Flask
if __name__ == '__main__':
    app.run(host='192.168.1.35', port=5000, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://192.168.1.35:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\hp\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import os
import pickle
import cv2
import numpy as np
from flask import Flask, request, jsonify

# Création de l'application Flask
app = Flask(__name__)

# Chemin pour enregistrer le modèle combiné
MODEL_PATH = "models/svm_face_recognition_with_pca.pkl"  # Chemin mis à jour
ANTI_SPOOF_MODEL_PATH = "models/anti_spoof_model.pkl"

# Fonction pour charger les modèles
def load_models():
    with open(MODEL_PATH, 'rb') as f:
        svm_model, pca, label_encoder = pickle.load(f)
    with open(ANTI_SPOOF_MODEL_PATH, 'rb') as f:
        anti_spoof_model = pickle.load(f)
    return svm_model, pca, label_encoder, anti_spoof_model

# Fonction pour détecter les attaques anti-spoofing
def detect_spoofing(image, anti_spoof_model):
    image_resized = cv2.resize(image, (64, 64))  # Adapter selon le modèle
    image_preprocessed = image_resized / 255.0
    image_preprocessed = np.expand_dims(image_preprocessed, axis=0)  # Ajouter une dimension batch
    prediction = anti_spoof_model.predict(image_preprocessed)
    return prediction[0] > 0.5  # Seuil à ajuster selon votre modèle

# Fonction pour prétraiter l'image
def preprocess_image(image):
    # Conversion en niveaux de gris (optionnel, selon vos besoins)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Normalisation des pixels
    normalized_image = gray_image / 255.0

    # Redimensionner l'image à une taille fixe pour correspondre au modèle
    resized_image = cv2.resize(normalized_image, (160, 160))

    # Flatten pour adapter aux modèles SVM (si non basé sur des convolutions)
    flattened_image = resized_image.flatten()

    return flattened_image

# Fonction pour prédire l'identité avec SVM et PCA
def recognize_face_with_svm(face_embedding, svm_model, pca, label_encoder):
    reduced_embedding = pca.transform([face_embedding])
    predicted_label_index = svm_model.predict(reduced_embedding)[0]
    predicted_label = label_encoder.inverse_transform([predicted_label_index])[0]
    probability = max(svm_model.predict_proba(reduced_embedding)[0])
    return predicted_label, probability

# Fonction de traitement des images envoyées
@app.route('/upload_frames', methods=['POST'])
def upload_frames():
    try:
        # Charger les modèles
        svm_model, pca, label_encoder, anti_spoof_model = load_models()

        # Récupération de l'image envoyée
        file = request.files['image']
        user_identity = request.form.get('identity')
        if not file:
            return jsonify({"error": "Aucune image n'a été envoyée."}), 400

        # Lecture de l'image
        img = cv2.imdecode(np.frombuffer(file.read(), np.uint8), cv2.IMREAD_COLOR)

        # Détection anti-spoofing
        if not detect_spoofing(img, anti_spoof_model):
            return jsonify({"error": "Détection de spoofing : image non valide."}), 403

        # Prétraitement de l'image
        face_embedding = preprocess_image(img)  # Application du prétraitement défini

        # Simuler un embedding (Remplacez cette ligne par l'appel à FaceNet)
        face_embedding = np.random.rand(128)  # Remplacez par la véritable extraction

        # Reconnaissance faciale
        label, probability = recognize_face_with_svm(face_embedding, svm_model, pca, label_encoder)

        # Vérification d'identité
        if user_identity and user_identity != label:
            return jsonify({"error": "Identité non vérifiée. Correspondance échouée."}), 401

        # Retour des résultats
        return jsonify({"label": label, "probability": probability})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)

In [1]:
import sqlite3
from datetime import datetime

# Connexion à la base de données SQLite (ajustez le chemin si nécessaire)
conn = sqlite3.connect('face_recognition.db')
cursor = conn.cursor()

# Obtenez la date d'aujourd'hui au format 'YYYY-MM-DD'
aujourdhui = datetime.now().strftime('%Y-%m-%d')

# Suppression des utilisateurs enregistrés avant aujourd'hui
query = "DELETE FROM user_data WHERE timestamp < ?"
cursor.execute(query, (aujourdhui,))

# Validation de la transaction
conn.commit()

# Fermeture de la connexion
conn.close()

print("Utilisateurs supprimés avec succès.")

Utilisateurs supprimés avec succès.


In [5]:
import cv2
def start_video_stream():
    # Replace this with the correct ESP32 stream URL
    ESP32_STREAM_URL = "http://192.168.137.83/capture"

    # Open video stream
    cap = cv2.VideoCapture(ESP32_STREAM_URL)

    # Check if the stream was successfully opened
    if not cap.isOpened():
        print("Error: Could not open video stream from ESP32.")
        return

    # Process video stream
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to read frame from ESP32.")
            break

        # Display the video feed
        cv2.imshow('ESP32 Video Stream', frame)

        # Exit on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()
start_video_stream()

Error: Could not open video stream from ESP32.


In [9]:
import http.client
import json

conn = http.client.HTTPSConnection("d9ydxr.api.infobip.com")
payload = json.dumps({
    "messages": [
        {
            "destinations": [{"to":"212688552708"}],
            "from": "ServiceSMS",
            "text": "Congratulations on sending your first message. Go ahead and check the delivery report in the next step."
        }
    ]
})
headers = {
    'Authorization': 'App afe5bc66d5b3be2cc4c2c52cd0d395c4-893a5145-1842-4c3a-be84-ff70d3ebfb42',
    'Content-Type': 'application/json',
    'Accept': 'application/json'
}
conn.request("POST", "/sms/2/text/advanced", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

{"messages":[{"messageId":"4345599663664335296466","status":{"description":"Message sent to next instance","groupId":1,"groupName":"PENDING","id":26,"name":"PENDING_ACCEPTED"},"to":"212688552708"}]}
